In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import date
from pathlib import Path
import time

# --- 1. 資料清洗模組 (Transformation) ---
def clean_and_prepare_data(df, rename_col, date_col):
    """
    專責處理資料清洗、格式統一與初步篩選
    """
    if df.empty:
        return df

    # A. 欄位補全：確保「發行性質」存在
    # B. 性質判定：根據「發行市場」內容補足「發行性質」
    # 解決 2016 年舊資料或部分 API 未提供性質的問題
    if "發行性質" not in df.columns:
        df.insert(loc=5, column="發行性質", value="")
        df["發行性質"] = np.where(
            df["發行市場"].str.contains("初上市", na=False), "初上市",
            np.where(df["發行市場"].str.contains("初上櫃", na=False), "初上櫃", "")
        )

    # C. 統一更名
    df = df.rename(columns=rename_col)

    # D. 強制日期轉換：將所有日期欄位轉為 Pandas Timestamp 物件
    for col in date_col:
        if col in df.columns:
            # errors='coerce' 會將無法解析的日期變成 NaT (不至於報錯中斷)
            df[col] = pd.to_datetime(df[col], format="%Y/%m/%d", errors="coerce")

    # E. 基本過濾：僅保留含有「初上市」或「初上櫃」字樣的資料
    # 使用 copy() 避免 SettingWithCopyWarning
    condition = df["發行性質"].str.contains("初上市|初上櫃", na=False)
    df = df[condition].copy()

    # F. 去除重複：確保「證券代號」與「開標日期」組合唯一
    # df = df.drop_duplicates(subset=["證券代號", "開標日期"], keep="last")

    return df

# --- 2. 爬取與主邏輯模組 (ETL Control) ---
def data_extract_manager(api_url, save_file_path, rename_col, date_col, year_range_url):
    today = pd.Timestamp(date.today())
    params = {"response": "json"}

    # A. 判斷模式：全量爬取 vs 增量更新
    if not save_file_path.exists():
        print(">>> [模式：初次全量] 檔案不存在，準備爬取歷史所有年份...")

        # 取得年份範圍
        res_years = requests.get(year_range_url, params=params)
        year_info = res_years.json()
        start_year = int(year_info.get("startYear", 2016))
        end_year = int(year_info.get("endYear", today.year))

        all_data_list = []
        for y in range(start_year, end_year + 1):
            print(f"正在抓取 {y} 年...")
            # 證交所 API 以 date=YYYY0101 參數切換年份
            req = requests.get(api_url, params={"date": f"{y}0101", "response": "json"})
            data_json = req.json()
            if "data" in data_json:
                temp_df = pd.DataFrame(data_json["data"], columns=data_json["fields"])
                # 執行清洗
                cleaned_df = clean_and_prepare_data(temp_df, rename_col, date_col)
                all_data_list.append(cleaned_df)
            time.sleep(1) # 禮貌爬蟲，避免被鎖 IP

        raw_df = pd.concat(all_data_list, ignore_index=True)

        # 全量模式過濾：僅保留已開標資料
        final_df = raw_df[raw_df["撥券日期(上市、上櫃日期)"] <= today]


    else:
        print(">>> [模式：增量更新] 讀取現有資料進行比對...")
        # 1. 讀取舊資料並取得最新基準日
        old_df = pd.read_csv(save_file_path)
        # 務必轉換舊資料日期以便比對
        old_df["撥券日期(上市、上櫃日期)"] = pd.to_datetime(old_df["撥券日期(上市、上櫃日期)"])
        max_date = old_df["撥券日期(上市、上櫃日期)"].max()
        print(f"資料庫最新日期為: {max_date.date()}")

        # 2. 抓取 API 當前最新資料
        req = requests.get(api_url, params=params)
        data_json = req.json()
        new_raw_df = pd.DataFrame(data_json["data"], columns=data_json["fields"])

        # 3. 執行清洗
        cleaned_new_df = clean_and_prepare_data(new_raw_df, rename_col, date_col)

        # 4. 核心增量條件：(日期 > 資料庫最新) 且 (日期 <= 今天)
        # 這會排除掉「還沒開標」的預告案
        incremental_df = cleaned_new_df[
            (cleaned_new_df["撥券日期(上市、上櫃日期)"] > max_date) &
            (cleaned_new_df["撥券日期(上市、上櫃日期)"] <= today)
        ]

        if not incremental_df.empty:
            print(f"檢測到 {len(incremental_df)} 筆新符合條件的資料，正在補進...")
            final_df = pd.concat([old_df, incremental_df], ignore_index=True)
        else:
            print("目前沒有符合條件的新資料。")
            return
    # B. 存檔輸出
    final_df.to_csv(save_file_path, index=False, encoding="utf-8-sig")
    print(f"任務完成！目前總筆數：{len(final_df)}")
    return final_df

    print(f"任務完成！目前總筆數：{len(final_df)}")


def main():
    # 設定存檔路徑 (Colab 環境建議)
    save_folder = Path("/content/drive/MyDrive/Colab Notebooks/stock_auction_pred_project/csv")
    save_folder.mkdir(exist_ok=True)
    file_path = save_folder / "rawdata.csv"

    # 參數定義
    config = {
        "api_url": "https://www.twse.com.tw/rwd/zh/announcement/auction",
        "year_range_url": "https://www.twse.com.tw/rwd/zh/announcement/auction/auctionYear",
        "rename_col": {
            "投標截止日": "投標結束日",
            "競拍股數": "競拍數量(張)",
            "最低每標單位(張)": "最低每標單投標數量(張)",
            "最高投(得)標單位(張)": "最高投(得)標數量(張)",
            "保證金成數": "保證金成數(%)",
            "合格投標量(張)": "合格投標數量(張)",
            "實際承銷價格(元)": "承銷價格(元)",
            "取消競價拍賣（流標或取消）": "取消競價拍賣(流標或取消)"
        },
        "date_col": ["開標日期", "投標開始日", "投標結束日", "撥券日期(上市、上櫃日期)"]
    }

    # 執行
    final_df = data_extract_manager(
        config["api_url"],
        file_path,
        config["rename_col"],
        config["date_col"],
        config["year_range_url"]
    )
# --- 3. 執行執行點 ---
if __name__ == "__main__":
    main()

>>> [模式：增量更新] 讀取現有資料進行比對...
資料庫最新日期為: 2026-01-16
目前沒有符合條件的新資料。


In [ ]:
from pathlib import Path

print(Path().resolve())

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/stock_auction_pred_project/csv/rawdata.csv")

print(df.columns)